In [1]:
import os, sys
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import torchvision
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import numpy as np
from PIL import Image
import torch.utils as utils
import matplotlib.pyplot as plt
from IPython.display import clear_output, display
lr = 0.001

In [2]:
import torchvision.models as  models
import torch.nn as nn
from torchsummary import summary

device = torch.device("cuda:0")
'''
net = models.resnet18(pretrained=False, progress=True)
net.fc = nn.Linear(  in_features=512,  out_features=10  )
net.conv1=nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
'''
net = models.MobileNetV2()
net.features[0][0]= nn.Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
net.classifier[1]=nn.Linear(in_features=1280, out_features=10, bias=True)

if torch.cuda.device_count() >1:
    net = nn.DataParallel(net)

net.to(device)

/home/xiyang/anaconda3/envs/test3/lib/python3.6/site-packages/torch/nn/parallel/data_parallel.py:26: UserWarning: 
    There is an imbalance between your GPUs. You may want to exclude GPU 0 which
    has less than 75% of the memory or cores of GPU 1. You can do so by setting
    the device_ids argument to DataParallel, or by setting the CUDA_VISIBLE_DEVICES
    environment variable.
  warnings.warn(imbalance_warn.format(device_ids[min_pos], device_ids[max_pos]))


DataParallel(
  (module): MobileNetV2(
    (features): Sequential(
      (0): ConvBNReLU(
        (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6(inplace=True)
      )
      (1): InvertedResidual(
        (conv): Sequential(
          (0): ConvBNReLU(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU6(inplace=True)
          )
          (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (2): InvertedResidual(
        (conv): Sequential(
          (0): ConvBNReLU(
            (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=Fals

In [3]:
transform = transforms.Compose([ transforms.Resize((224,224)),  transforms.ToTensor(), \
                                transforms.Normalize((0.1307,), (0.3081,))])
mnist_test  = datasets.MNIST('./data', train=False, download=True, transform=transform)
mnist_testloader  = utils.data.DataLoader(mnist_test,  batch_size=1,  shuffle=False, num_workers=2)

In [4]:
def fgsm_attack(image, epsilon, data_grad):

    sign_data_grad = data_grad.sign()

    perturbed_image = image + epsilon*sign_data_grad

    return perturbed_image   

In [5]:
def attack_model(net, dataloader, epsilon, Pretrained=None):
    if not Pretrained is None:
        net.load_state_dict(torch.load(Pretrained))
    correct = 0
    uncorrect = 0
    adv_samples   = []
    sample_labels = []
    for _, data in enumerate(dataloader):
        image, target = data
        image, target = image.to(device), target.to(device)

        image.requires_grad = True
        target = torch.LongTensor(list(target)).to(device)
        output = net(image)

        _, pred_cls = torch.max(output, 1)
        
        if pred_cls != target:   # ignore the wrong  predictions
            uncorrect +=1
            continue
 
        loss = F.nll_loss(output, target)

        net.zero_grad()

        loss.backward()

        image_grad = image.grad.data

        perturbed_image = fgsm_attack(image, epsilon, image_grad)

        adv_samples.append(perturbed_image.detach().cpu())
        
        sample_labels.append(target)

        output = net(perturbed_image)

        _, pred_cls = torch.max(output, 1)
        correct += (pred_cls == target).float().sum().item()
        
    print(uncorrect)
    print(correct)
    print(len(dataloader))
  
    final_acc = correct/float(len(dataloader))
    print("Epsilon: {}\tTest Accuracy = {}".format(epsilon, final_acc))

    return adv_samples, sample_labels

In [6]:
Pretrained ='/home/xiyang/Desktop/model_for_report4/MB/MB_model_60.pt' 
net.load_state_dict(torch.load(Pretrained))
net.eval()
epsilonlist = [0, 0.05, 0.1, 0.2, 0.4, 0.6]
for e in epsilonlist: 
    adv_samples, sample_labels = attack_model(net, mnist_testloader, epsilon=e, Pretrained=Pretrained)


497
9503.0
10000
Epsilon: 0	Test Accuracy = 0.9503
497
2160.0
10000
Epsilon: 0.05	Test Accuracy = 0.216
497
960.0
10000
Epsilon: 0.1	Test Accuracy = 0.096
497
847.0
10000
Epsilon: 0.2	Test Accuracy = 0.0847
497
847.0
10000
Epsilon: 0.4	Test Accuracy = 0.0847
497
847.0
10000
Epsilon: 0.6	Test Accuracy = 0.0847
